In [1]:
!pip install datasets transformers scikit-learn pandas


In [2]:
from datasets import load_dataset
import pandas as pd

# Load dataset
dataset = load_dataset("banking77")

# Convert train and test to pandas
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

# Quick look
print("Training set size:", len(df_train))
print("Test set size:", len(df_test))
print(df_train.head())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

Training set size: 10003
Test set size: 3080
                                                text  label
0                     I am still waiting on my card?     11
1  What can I do if my card still hasn't arrived ...     11
2  I have been waiting over a week. Is the card s...     11
3  Can I track my card while it is in the process...     11
4  How do I know if I will get my card, or if it ...     11


In [3]:
# Select features and target
X_train = df_train['text']
y_train = df_train['label']

X_test = df_test['text']
y_test = df_test['label']


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit on training data and transform
X_train_vec = vectorizer.fit_transform(X_train)

# Transform test data
X_test_vec = vectorizer.transform(X_test)


In [5]:
from sklearn.linear_model import LogisticRegression

# Create the model
model = LogisticRegression(max_iter=300)

# Train on vectorized training data
model.fit(X_train_vec, y_train)


LogisticRegression(max_iter=300)

In [6]:
from sklearn.metrics import classification_report

# Predict on test data
y_pred = model.predict(X_test_vec)

# Show performance metrics
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.90      0.95        40
           1       0.93      0.97      0.95        40
           2       0.97      0.97      0.97        40
           3       0.90      0.93      0.91        40
           4       0.97      0.90      0.94        40
           5       0.64      0.75      0.69        40
           6       0.85      0.88      0.86        40
           7       0.89      0.82      0.86        40
           8       0.89      0.97      0.93        40
           9       1.00      0.97      0.99        40
          10       0.84      0.53      0.65        40
          11       0.75      0.90      0.82        40
          12       0.75      0.82      0.79        40
          13       0.86      0.93      0.89        40
          14       0.63      0.82      0.72        40
          15       0.78      0.90      0.84        40
          16       0.64      0.72      0.68        40
          17       0.90    

In [7]:
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
sample_ticket = "My card got blocked and I can't make a payment"

# Get sentiment
sentiment_result = sentiment_analyzer(sample_ticket)[0]
print(sentiment_result)

{'label': 'NEGATIVE', 'score': 0.9997414946556091}


In [9]:
sample_ticket = "My experience is good with the card usage"

# Get sentiment
sentiment_result = sentiment_analyzer(sample_ticket)[0]
print(sentiment_result)

{'label': 'POSITIVE', 'score': 0.9987269043922424}


In [10]:
from transformers import pipeline

# Load text generation model
response_generator = pipeline("text2text-generation", model="t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [28]:
def generate_response(ticket_text, category_name, sentiment_label):
    if sentiment_label == "NEGATIVE":
        response = f"Thank you for contacting us regarding '{category_name}'. We understand your concern. Our team will assist you shortly. (Customer mood: {sentiment_label})"
    elif sentiment_label == "NEUTRAL":
        response = f"We have received your message regarding '{category_name}'. Thank you for reaching out. (Customer mood: {sentiment_label})"
    elif sentiment_label == "POSITIVE":
        response = f"Thank you for your feedback regarding '{category_name}'! We're glad everything is going well. (Customer mood: {sentiment_label})"
    else:
        response = f"Thank you for contacting us regarding '{category_name}'. (Customer mood: {sentiment_label})"
    return response


In [24]:
label_map = dict(zip(range(len(dataset['train'].features['label'].names)),
                     dataset['train'].features['label'].names))

In [29]:
def support_assistant(ticket_text):
    # 1️⃣ Predict category
    category_num= model.predict(vectorizer.transform([ticket_text]))[0]

    # Map to category name
    category_name = label_map[category_num]

    # 2️⃣ Predict sentiment
    sentiment_result = sentiment_analyzer(ticket_text)[0]
    sentiment_label = sentiment_result['label']

    # 3️⃣ Generate auto-response (template-based)
    response = generate_response(ticket_text, category_name, sentiment_label)

    # Return all results
    return {
        "Ticket Text": ticket_text,
        "Predicted Category": category_name,
        "Sentiment": sentiment_label,
        "AI Response": response
    }


In [30]:
ticket = "My card got blocked and I can't make a payment"
result = support_assistant(ticket)
print(result)

{'Ticket Text': "My card got blocked and I can't make a payment", 'Predicted Category': 'card_payment_not_recognised', 'Sentiment': 'NEGATIVE', 'AI Response': "Thank you for contacting us regarding 'card_payment_not_recognised'. We understand your concern. Our team will assist you shortly. (Customer mood: NEGATIVE)"}


In [31]:
ticket = "Thank you for resolving my account issue so quickly!"
result = support_assistant(ticket)
print(result)

{'Ticket Text': 'Thank you for resolving my account issue so quickly!', 'Predicted Category': 'reverted_card_payment?', 'Sentiment': 'POSITIVE', 'AI Response': "Thank you for your feedback regarding 'reverted_card_payment?'! We're glad everything is going well. (Customer mood: POSITIVE)"}


In [32]:
while True:
    ticket = input("Enter a customer ticket (or type 'exit' to quit): ")
    if ticket.lower() == "exit":
        print("Exiting support assistant.")
        break
    result = support_assistant(ticket)
    print("\nPredicted Category:", result['Predicted Category'])
    print("Sentiment:", result['Sentiment'])
    print("AI Response:", result['AI Response'])
    print("\n" + "-"*50 + "\n")

Enter a customer ticket (or type 'exit' to quit): The experience is good though having some issues with payments but not too much

Predicted Category: unable_to_verify_identity
Sentiment: POSITIVE
AI Response: Thank you for your feedback regarding 'unable_to_verify_identity'! We're glad everything is going well. (Customer mood: POSITIVE)

--------------------------------------------------

Enter a customer ticket (or type 'exit' to quit): exit
Exiting support assistant.
